# Airbyte Gong (已弃用)

请注意：此特定连接器加载器已弃用。请改用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个数据集成平台，用于从 API、数据库和文件到数据仓库和数据湖的 ELT 管道。它拥有最大的数据仓库和数据库 ELT 连接器目录。

此加载器将 Gong 连接器公开为文档加载器，允许您将各种 Gong 对象加载为文档。

## 安装

首先，你需要安装 `airbyte-source-gong` Python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-gong

## 示例

请查看 [Airbyte 文档页面](https://docs.airbyte.com/integrations/sources/gong/) 以了解如何配置读取器。

配置对象应遵循的 JSON schema 可以在 Github 上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-gong/source_gong/spec.yaml](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-gong/source_gong/spec.yaml)。

大致结构如下：
```python
{
  "access_key": "<access key name>",
  "access_key_secret": "<access key secret>",
  "start_date": "<date from which to start retrieving records from in ISO format, e.g. 2020-10-20T00:00:00Z>",
}
```

默认情况下，所有字段都作为元数据存储在文档中，文本内容为空字符串。通过转换读取器返回的文档来构建文档的文本内容。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteGongLoader

config = {
    # your gong configuration
}

loader = AirbyteGongLoader(
    config=config, stream_name="calls"
)  # check the documentation linked above for a list of all streams

现在您可以像往常一样加载文档了

In [ ]:
docs = loader.load()

由于 `load` 返回的是一个列表，它会一直阻塞，直到所有文档都加载完毕。为了更好地控制这个过程，您也可以使用 `lazy_load` 方法，它会返回一个迭代器：

In [ ]:
docs_iterator = loader.lazy_load()

请记住，默认情况下页面内容为空，metadata 对象包含来自记录的所有信息。要处理文档，请创建一个继承自基类的加载器类，并自行实现 `_handle_records` 方法：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteGongLoader(
    config=config, record_handler=handle_record, stream_name="calls"
)
docs = loader.load()

## 增量加载

某些流支持增量加载，这意味着源会跟踪已同步的记录，并且不会再次加载它们。这对于数据量大且频繁更新的源非常有用。

要利用此功能，请存储加载器的 `last_state` 属性，并在重新创建加载器时将其传入。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteGongLoader(
    config=config, stream_name="calls", state=last_state
)

new_docs = incremental_loader.load()